<a href="https://colab.research.google.com/github/SoraHoang/Classification-of-Echocardiographic-Images/blob/main/ClassificationOfEchocardiographicImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Thêm các thư viện**

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader

from torchvision import transforms
import torchvision

import matplotlib.pyplot as plt

from collections import namedtuple

from sklearn.metrics import classification_report

# **Sử dụng data đã được lưu trên google drive**

In [ ]:
train_path = './drive/MyDrive/Data/DATA_CHAMBER_2021/train'
test_path = './drive/MyDrive/Data/DATA_CHAMBER_2021/test'

# **Chuẩn bị dữ liệu**

* Tạo namedtuple TrainTest cho tiện thao tác

In [ ]:
TrainTest = namedtuple('TrainTest', ['train', 'test'])

* 3 classes: {2C, 3C, 4C}

In [ ]:
def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

Chuẩn bị dữ liệu

* Đọc dữ liệu từ đường dẫn train_path và test_path
* Kích thước không đồng bộ => ảnh cần rescale 224x224x3
* Đưa dữ liệu ảnh về dạng tensor => 3x224x224

In [ ]:
def prepare_data():
  image_resize = 224

  transform_train = transforms.Compose([
    transforms.Resize((image_resize, image_resize)),
    transforms.ToTensor()
  ])
  transform_test = transforms.Compose([
    transforms.Resize((image_resize, image_resize)),
    transforms.ToTensor()
  ])

  trainset = torchvision.datasets.ImageFolder(root=train_path, transform=transform_train)
  testset = torchvision.datasets.ImageFolder(root=test_path, transform=transform_test)

  return TrainTest(train=trainset, test=testset)

Chuẩn bị loader để chia batch dữ liệu

*   batch_size = 32
*   num_workers = 4



In [ ]:
def prepare_loader(datasets):
  batch_size = 32
  num_workers = 4

  trainloader = DataLoader(dataset=datasets.train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
  testloader = DataLoader(dataset=datasets.test, batch_size=batch_size, shuffle=False, num_workers=num_workers)

  return TrainTest(train=trainloader, test = testloader)

# **Xử lý các epoch**

Train trong mỗi epoch

In [ ]:
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0
  reporting_steps = 60
  for i, (images, labels) in enumerate(loader):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % reporting_steps == reporting_steps-1:
      print(f"Epoch {epoch} step {i} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0

Test trong mỗi epoch

In [ ]:
def test_epoch(epoch, model, loader, device):
  ytrue = []
  ypred = []
  with torch.no_grad():
    model.eval()
    
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)

      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())

  return ypred, ytrue

# **Xây dựng và thực nghiệm mô hình**

In [ ]:
def main(PATH='./model.pth', MODEL=None):
  classes = get_classes()
  datasets = prepare_data()
  # print('train', len(datasets.train), 'test', len(datasets.test))
  # img, label = datasets.train[0]
  # print(img.shape)

  loaders = prepare_loader(datasets)
  device = torch.device("cuda:0")

  if MODEL == 'vgg16':
    model = torchvision.models.vgg16()
    model.classifier[6] = torch.nn.modules.linear.Linear(in_features=4096, out_features=3, bias=True)
  else:
    pass

  model.to(device)
  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

  for epoch in range(10):
    train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    ypred, ytrue = test_epoch(epoch, model, loaders.test, device)
    print(classification_report(ytrue, ypred, target_names=classes))

    torch.save(model.state_dict(), PATH)

  return model

Thử mô hình VGG16

In [ ]:
model = main(PATH='./vgg16.pth', MODEL='vgg16')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch 0 step 59 ave_loss 0.9768
Epoch 0 step 119 ave_loss 0.6065
Epoch 0 step 179 ave_loss 0.4853
              precision    recall  f1-score   support

          2C       0.76      0.81      0.78       409
          3C       0.65      0.94      0.77       367
          4C       1.00      0.77      0.87       831

    accuracy                           0.82      1607
   macro avg       0.80      0.84      0.81      1607
weighted avg       0.86      0.82      0.82      1607

Epoch 1 step 59 ave_loss 0.2033
Epoch 1 step 119 ave_loss 0.1023
Epoch 1 step 179 ave_loss 0.0554
              precision    recall  f1-score   support

          2C       0.94      0.85      0.89       409
          3C       0.85      0.98      0.91       367
          4C       1.00      0.97      0.99       831

    accuracy                           0.94      1607
   macro avg       0.93      0.94      0.93      1607
weighted avg       0.95      0.94      0.94      1607

Epoch 2 step 59 ave_loss 0.0257
Epoch 2 st